In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [2]:
%load_ext autoreload
%autoreload 2
from bnpm import server
import bnpm

In [3]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc

In [4]:
import numpy as np
import natsort

------------
# ==== Start ====

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = False

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username:  rh183
Password:  ········


## Prepare `sftp` and `ssh` objects

In [6]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)

In [7]:
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [8]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [9]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [10]:
prompt_snip = f'[{username}'

In [11]:
send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

# Batch run

In [12]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/',
    search_pattern_re='cam4.*avi',
    max_depth=5,
    find_folders=False,
    find_files=True,
    verbose=True,
))

/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230804/cam42023-08-04T12_06_21.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230726/cam42023-07-26T11_49_44.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230708/cam42023-07-08T10_28_34.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230808/cam42023-08-08T13_36_08.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230711/cam42023-07-11T10_48_34.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230725/cam42023-07-25T11_19_05.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230721/cam42023-07-21T11_21_03.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230722/cam4

In [13]:
dates_toUse = natsort.natsorted([Path(p).parts[-2] for p in paths_found])
dates_toUse

['20230703',
 '20230705',
 '20230706',
 '20230707',
 '20230708',
 '20230709',
 '20230710',
 '20230711',
 '20230712',
 '20230713',
 '20230714',
 '20230715',
 '20230716',
 '20230717',
 '20230718',
 '20230719',
 '20230720',
 '20230721',
 '20230722',
 '20230723',
 '20230724',
 '20230725',
 '20230726',
 '20230727',
 '20230728',
 '20230729',
 '20230730',
 '20230731',
 '20230801',
 '20230802',
 '20230803',
 '20230804',
 '20230805',
 '20230806',
 '20230807',
 '20230808',
 '20230809',
 '20230811']

In [14]:
paths_vids = {d: v for d, v in zip(dates_toUse, paths_found)}

## Start a small interactive node

In [18]:
send_expect(f'srun --pty -p interactive -t 0-00:30:00 --mem=4G /bin/bash');

AP]$ srun: Job step aborted: Waiting up to 62 seconds for job step to finish. NBA
slurmstepd: error: *** STEP 16991840.0 ON compute-a-16-160 CANCELLED AT 2023-09-06T16:46:21 DUE TO TIME LIMIT ***
srun: error: compute-a-16-160: task 0: Killed
(base) [rh183@login06 ~]$ 


## Pull/update repo

In [19]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP')
send_expect('git pull');

bash --pty -p interactive -t 0-00:30:00 --mem=4G /bin/ 
srun: job 17003106 queued and waiting for resources

cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP

srun: job 17003106 has been allocated resources

cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP

(base) [rh183@compute-a-16-170 ~]$ 
_repos/NBAP/hms/neurobio/sabatini/rich/github 
(base) [rh183@compute-a-16-170 NBAP]$ 


## Activate environment

In [20]:
name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'

send_expect(f'conda activate {name_env}');

git pull

conda activate /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Already up-to-date.

ini/rich/virtual_envs/FR16-170 NBAP]$ conda activate /n/data1/hms/neurobio/sabat 



## Prepare commands

In [22]:
paths_vids.keys()

dict_keys(['20230703', '20230705', '20230706', '20230707', '20230708', '20230709', '20230710', '20230711', '20230712', '20230713', '20230714', '20230715', '20230716', '20230717', '20230718', '20230719', '20230720', '20230721', '20230722', '20230723', '20230724', '20230725', '20230726', '20230727', '20230728', '20230729', '20230730', '20230731', '20230801', '20230802', '20230803', '20230804', '20230805', '20230806', '20230807', '20230808', '20230809', '20230811'])

In [30]:
duration_sessions = {date: 60*60 if float(date) < float(20230725) else 80*60 for date in dates_toUse}

In [32]:
for path_vid in paths_found[2:]:

    mouse = Path(path_vid).parts[-4]
    date = Path(path_vid).parts[-2]
    
    name_slurm = mouse[-3:] + '_' + date[-4:]

    path_vid = path_vid
    path_mask = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction') / mouse / date / ('mask_' + Path(path_vid).stem + '.npy') )

    dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction') / mouse / date )

    name_job = 'jobNum_'


    path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

    path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
    path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server

    ## Expectation for dispatch args: 
    # path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env

    time_fastForward = duration_sessions[date] - 1
    
    commands = {
    #     'make_dir': f"mkdir -p {dir_data_remote}",
    #     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    #     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
    'dispatch': f"\
    python3 \
    {path_dispatcher_remote} \
    {path_script_remote} \
    {dir_saveOutputs} \
    {path_vid} \
    {path_mask} \
    {name_job} \
    {name_slurm} \
    {name_env}\
    {time_fastForward}\
    ",
    }
    display(commands)
    
    # sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
    # sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));
    
    # send_expect(commands['dispatch']);

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230706/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230706     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230706/cam42023-07-06T11_48_27.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230706/mask_cam42023-07-06T11_48_27.npy     jobNum_     03L_0706     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230707/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230707     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230707/cam42023-07-07T10_15_25.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230707/mask_cam42023-07-07T10_15_25.npy     jobNum_     03L_0707     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230708/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230708     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230708/cam42023-07-08T10_28_34.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230708/mask_cam42023-07-08T10_28_34.npy     jobNum_     03L_0708     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230709/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230709     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230709/cam42023-07-09T10_22_38.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230709/mask_cam42023-07-09T10_22_38.npy     jobNum_     03L_0709     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230710/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230710     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230710/cam42023-07-10T10_46_20.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230710/mask_cam42023-07-10T10_46_20.npy     jobNum_     03L_0710     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

...

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230806/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230806     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230806/cam42023-08-06T13_36_28.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230806/mask_cam42023-08-06T13_36_28.npy     jobNum_     03L_0806     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    4799    '}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230807/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230807     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230807/cam42023-08-07T12_12_09.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230807/mask_cam42023-08-07T12_12_09.npy     jobNum_     03L_0807     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    4799    '}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230808/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230808     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230808/cam42023-08-08T13_36_08.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230808/mask_cam42023-08-08T13_36_08.npy     jobNum_     03L_0808     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    4799    '}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230809/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230809     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230809/cam42023-08-09T11_53_04.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230809/mask_cam42023-08-09T11_53_04.npy     jobNum_     03L_0809     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    4799    '}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230811/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230811     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403L/camera_data/20230811/cam42023-08-11T12_38_36.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403L/20230811/mask_cam42023-08-11T12_38_36.npy     jobNum_     03L_0811     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    4799    '}

### check on job

### Check if it worked

In [85]:
test = natsort.natsorted(sftp.search_recursive(
    path=r'/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R',
    search_pattern_re=r'idx_eye_laser.pkl',
))
display(test)

/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230531/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230420/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230523/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230517/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230515/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230430/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230512/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230511/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/anal

['/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230419/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230420/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230424/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230425/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230426/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230427/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230428/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230429/jobNum_0/idx_eye_laser.pkl',
 '/n/dat

In [86]:
len(test)

37

In [73]:
len(paths_found)

38

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()